In [ ]:
%load_ext autoreload

In [ ]:
%autoreload # When utils.py is updated
from utils_siamese import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
root_path = '/your_directory' 
print(root_path)

In [ ]:
# Define data type (L8-Landsat8, S2-Sentinel2, S1-Sentinel1)
img_type = 'L8'

if img_type = 'L8':
    # Load images
    ref_2019 = load_tif_image(root_path+'New_Images/References/res_10m/r10m_def_2019.tif').astype('float32') # actual 2019
    opt_2018 = load_tif_image(root_path+'New_Images/Landsat8/'+'cut_land8_2018.tif').astype('float32')   
    opt_2019 = load_tif_image(root_path+'New_Images/Landsat8/'+'cut_land8_2019.tif').astype('float32')

    # Resize images
    opt_2018 = resize_image(opt_2018.copy(), ref_2019.shape[0], ref_2019.shape[1])
    opt_2019 = resize_image(opt_2019.copy(), ref_2019.shape[0], ref_2019.shape[1])  

    # Filter outliers
    opt_2018 = filter_outliers(opt_2018.copy()) 
    opt_2019 = filter_outliers(opt_2019.copy())
    
    image_stack = np.concatenate((opt_2018, opt_2019), axis=-1)
    print('landsat_resize:', image_stack.shape)
    del opt_2018, opt_2019

if img_type = 'S2':
    # Load images
    sent2_2018_1 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2018_10m_b2348.tif').astype('float32')
    sent2_2018_2 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2018_20m_b5678a1112.tif').astype('float32')
    
    # Resize bands of 20m
    sent2_2018_2 = resize_image(sent2_2018_2.copy(), sent2_2018_1.shape[0], sent2_2018_1.shape[1])
    sent2_2018 = np.concatenate((sent2_2018_1, sent2_2018_2), axis=-1)
    del sent2_2018_1, sent2_2018_2
    
    sent2_2019_1 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2019_10m_b2348.tif').astype('float32')
    sent2_2019_2 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2019_20m_b5678a1112.tif').astype('float32')   
    
    # Resize bands of 20m
    sent2_2019_2 = resize_image(sent2_2019_2.copy(), sent2_2019_1.shape[0], sent2_2019_1.shape[1])
    sent2_2019 = np.concatenate((sent2_2019_1, sent2_2019_2), axis=-1)
    del sent2_2019_1, sent2_2019_2
    
    # Filter outliers
    sent2_2018 = filter_outliers(sent2_2018.copy()) 
    sent2_2019 = filter_outliers(sent2_2019.copy()) 
    
    image_stack = np.concatenate((sent2_2018, sent2_2019), axis=-1)
    print('Image stack:', image_stack.shape)
    del sent2_2018, sent2_2019

if img_type = 'S1':
    # Load images
    sar_2018_vh = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vh_2018.tif').astype('float32'), axis = -1)
    sar_2018_vv = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vv_2018.tif').astype('float32'), axis = -1)
    sar_2019_vh = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vh_2019.tif').astype('float32'), axis = -1)
    sar_2019_vv = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vv_2019.tif').astype('float32'), axis = -1)
    
    sar_2018 = np.concatenate((sar_2018_vh, sar_2018_vv), axis=-1)
    sar_2019 = np.concatenate((sar_2019_vh, sar_2019_vv), axis=-1)
    del sar_2018_vh, sar_2018_vv, sar_2019_vh, sar_2019_vv
    
    # Filter outliers
    sar_2018 = filter_outliers(sar_2018.copy()) 
    sar_2019 = filter_outliers(sar_2019.copy()) 

    image_stack = np.concatenate((sar_2018, sar_2019), axis=-1)
    print('Image stack:', image_stack.shape)
    del sar_2018, sar_2019

# load references     
# Load current reference 
ref_2019 = load_tif_image(root_path+'New_Images/References/res_10m/r10m_def_2019.tif').astype('float32') # actual 2019
# Load past references
#past_ref = np.load(root_path+'New_Images/References/past_ref_and_clouds.npy').astype('float32')
past_ref1 = load_tif_image(root_path+'New_Images/References/res_10m/r10m_def_1988_2007.tif').astype('float32') # 1988_2007
past_ref2 = load_tif_image(root_path+'New_Images/References/res_10m/r10m_def_2008_2018.tif').astype('float32') # 2008_2018
clouds_2018 = load_tif_image(root_path+'New_Images/References/cut_b10_2018.tif').astype('float32')
clouds_2018 = resize_image(np.expand_dims(clouds_2018.copy(), axis = -1), ref_2019.shape[0], ref_2019.shape[1])
clouds_2018 = binary_mask_cloud(clouds_2018.copy(), 50)
clouds_2019 = load_tif_image(root_path+'New_Images/References/cut_b10_2019.tif').astype('float32') 
clouds_2019 = resize_image(np.expand_dims(clouds_2019.copy(), axis = -1), ref_2019.shape[0], ref_2019.shape[1])
clouds_2019 = binary_mask_cloud(clouds_2019.copy(), 50)

In [ ]:
# Create label mask
past_ref = past_ref1 + past_ref2 + clouds_2018 + clouds_2019
past_ref[past_ref>=1] = 1
buffer = 2
final_mask1 = mask_no_considered(ref_2019, buffer, past_ref)
del past_ref1, past_ref2, clouds_2018, clouds_2019

lim_x = 10000
lim_y = 7000
image_stack = image_stack[:lim_x, :lim_y, :]
final_mask1 = final_mask1[:lim_x, :lim_y]
ref_2019 = ref_2019[:lim_x, :lim_y]

h_, w_, channels = image_stack.shape
print('image stack size: ', image_stack.shape)

# Normalization
type_norm = 1
image_array = normalization(image_stack.copy(), type_norm)
print(np.min(image_array), np.max(image_array))
del image_stack

# Print pertengate of each class (whole image)
print('Total no-deforestaion class is {}'.format(len(final_mask1[final_mask1==0])))
print('Total deforestaion class is {}'.format(len(final_mask1[final_mask1==1])))
print('Total past deforestaion class is {}'.format(len(final_mask1[final_mask1==1])))
print('Percentage of deforestaion class is {:.2f}'.format((len(final_mask1[final_mask1==1])*100)/len(final_mask1[final_mask1==0])))

In [ ]:
# Create tile mask
mask_tiles = create_mask(final_mask1.shape[0], final_mask1.shape[1], grid_size=(5, 4))
image_array = image_array[:mask_tiles.shape[0], :mask_tiles.shape[1],:]
final_mask1 = final_mask1[:mask_tiles.shape[0], :mask_tiles.shape[1]]

print('mask: ',mask_tiles.shape)
print('image stack: ', image_array.shape)
print('ref :', final_mask1.shape)
#plt.imshow(mask_tiles)

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(final_mask1, cmap = 'jet')

In [ ]:
# Define tiles for training, validation, and test sets
tiles_tr = [1,3,5,8,11,13,14,20]
tiles_val = [6,19]
tiles_ts = (list(set(np.arange(20)+1)-set(tiles_tr)-set(tiles_val)))

mask_tr_val = np.zeros((mask_tiles.shape)).astype('float32')
# Training and validation mask
for tr_ in tiles_tr:
    mask_tr_val[mask_tiles == tr_] = 1

for val_ in tiles_val:
    mask_tr_val[mask_tiles == val_] = 2

mask_amazon_ts = np.zeros((mask_tiles.shape)).astype('float32')
for ts_ in tiles_ts:
    mask_amazon_ts[mask_tiles == ts_] = 1

In [ ]:
# Create ixd image to extract patches
overlap = 0.7
patch_size = 128
batch_size = 32
im_idx = create_idx_image(final_mask1)
patches_idx = extract_patches(im_idx, patch_size=(patch_size, patch_size), overlap=overlap).reshape(-1,patch_size, patch_size)
patches_mask = extract_patches(mask_tr_val, patch_size=(patch_size, patch_size), overlap=overlap).reshape(-1, patch_size, patch_size)
del im_idx

In [ ]:
# Selecting index trn val and test patches idx
idx_trn = np.squeeze(np.where(patches_mask.sum(axis=(1, 2))==patch_size**2))
idx_val = np.squeeze(np.where(patches_mask.sum(axis=(1, 2))==2*patch_size**2))

patches_idx_trn = patches_idx[idx_trn]
patches_idx_val = patches_idx[idx_val]

print('Number of training patches:  ', len(patches_idx_trn), 'Number of validation patches', len(patches_idx_val))

In [ ]:
X_train = retrieve_idx_percentage(final_mask1, patches_idx_trn, pertentage = 2)
X_valid = retrieve_idx_percentage(final_mask1, patches_idx_val, pertentage = 2)
print(X_train.shape, X_valid.shape)
del patches_idx_trn, patches_idx_val

In [ ]:
def batch_generator(batches, image, reference, target_size, number_class, n_bands):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    image = image.reshape(-1, image.shape[-1])
    reference = reference.reshape(final_mask1.shape[0]*final_mask1.shape[1])
    while True:
        batch_x, batch_y = next(batches)
        batch_x = np.squeeze(batch_x.astype('int64'), axis = -1)
        #print(batch_x.shape)
        batch_img = np.zeros((batch_x.shape[0], target_size, target_size, image.shape[-1]))
        batch_ref = np.zeros((batch_x.shape[0], target_size, target_size, 1))
        
        for i in range(batch_x.shape[0]):
            if np.random.rand()>0.5:
                batch_x[i] = np.rot90(batch_x[i], 1)
            batch_img[i] = image[batch_x[i]] 
            #batch_ref[i] = tf.keras.utils.to_categorical(reference[batch_x[i]] , number_class)
            batch_ref[i] = reference[np.expand_dims(batch_x[i], axis = -1)]
                       
        yield batch_img[:,:,:,:n_bands//2], batch_img[:,:,:,n_bands//2:], batch_ref

def plot_images(img_t0, img_t1, distance, reference, figsize=(10, 5)):
    fig = plt.figure(figsize=figsize)
    x0 = img_t0[0,:,:,3:0:-1]
    x0 = (x0-np.min(x0))/(np.max(x0)-np.min(x0))
    x1 = img_t1[0,:,:,3:0:-1]
    x1 = (x1-np.min(x1))/(np.max(x1)-np.min(x1))

    ax1 = fig.add_subplot(141)
    plt.title('Image T0')
    ax1.imshow(x0)
    ax1.axis('off')

    ax2 = fig.add_subplot(142)
    plt.title('Image T1')
    ax2.imshow(x1)
    ax2.axis('off')

    ax3 = fig.add_subplot(143)
    plt.title('Distance')
    ax3.imshow(distance[0,:,:])
    ax3.axis('off')
    
    ax4 = fig.add_subplot(144)
    plt.title('Reference')
    ax4.imshow(reference[0,:,:])
    ax4.axis('off')

    plt.show()
    # fig.savefig(save_img_path+name+'_img_pt_br_2_elastic_'+str(epoch))
    plt.close() 

train_datagen = ImageDataGenerator(horizontal_flip = True,
                                   vertical_flip = True)
valid_datagen = ImageDataGenerator(horizontal_flip = True, 
                                   vertical_flip = True)

y_train = np.zeros((len(X_train)))
y_valid = np.zeros((len(X_valid)))

train_gen = train_datagen.flow(np.expand_dims(X_train, axis = -1), y_train,
                              batch_size=batch_size,
                              shuffle=True)

valid_gen = valid_datagen.flow(np.expand_dims(X_valid, axis = -1), y_valid,
                              batch_size=batch_size,
                              shuffle=False)

number_class = 3
train_gen_batch = batch_generator(train_gen, image_array, final_mask1, patch_size, number_class, channels)
valid_gen_batch = batch_generator(valid_gen, image_array, final_mask1, patch_size, number_class, channels)


In [ ]:
exp = 3
path_exp = root_path+'experiments/exp'+str(exp)
path_models = path_exp+'/models'
path_maps = path_exp+'/pred_maps'

if not os.path.exists(path_exp):
    os.makedirs(path_exp)   
if not os.path.exists(path_models):
    os.makedirs(path_models)   
if not os.path.exists(path_maps):
    os.makedirs(path_maps)
    
name = 'siamese'
times = 5

In [ ]:
metrics_all = []

## computation graph
K.clear_session()
# parameters

input_shape_enc = (patch_size, patch_size, channels//2)
input_shape_dec = (patch_size, patch_size, 1)

img_size, img_size, cdims = input_shape_enc
lr = 1e-3

# network definition
encoder = build_base_encoder(input_shape_enc)
encoder.summary()
decoder = build_base_decoder(input_shape_dec)
decoder.summary()

img_a = K.placeholder(dtype=tf.float32, name="img_a", shape=(batch_size, img_size, img_size, cdims))
img_b = K.placeholder(dtype=tf.float32, name="img_b", shape=(batch_size, img_size, img_size, cdims))
y_true = K.placeholder(dtype=tf.float32, name="y_true", shape=(batch_size, img_size, img_size, 1))

[feat1_a, feat2_a, feat3_a, feat4_a] = encoder(img_a)
[feat1_b, feat2_b, feat3_b, feat4_b] = encoder(img_b)

distance1 = euclidean_distance(feat4_a, feat4_b)

distance = decoder([distance1, feat1_a, feat2_a, feat3_a, feat1_b, feat2_b, feat3_b])

loss = contrastive_loss(y_true, distance, weights)

t_vars = tf.trainable_variables()
net_vars = [var for var in t_vars if '_net' in var.name]
optim = tf.train.AdamOptimizer(lr).minimize(loss, var_list=net_vars)

sess = K.get_session()

init_op = tf.variables_initializer(net_vars)

# **************
for tm in range(0,times):
    print('time: ', tm)
    
    sess.run(tf.global_variables_initializer())
    # print(net_vars)

    num_of_trn_batches = len(X_train) // batch_size
    num_of_val_batches = len(X_valid) // batch_size
    epochs = 100
    best_val_loss = np.inf
    # train the model
   
    for epoch in range(epochs):
        print('epoch :', epoch)
        trn_loss, trn_acc, t_dist = [], [], []
        start_time = time.time()
        for idx in range(0, num_of_trn_batches):

            # selecting a batch of images
            batch_t0, batch_t1, batch_ref = next(train_gen_batch)
            if batch_t0.shape[0] != batch_size:
                continue

            feed_dict={img_a: batch_t0, img_b: batch_t1, y_true: batch_ref}

            sess.run([optim], feed_dict=feed_dict)

            with sess.as_default():
                t_loss = loss.eval(feed_dict)
                #print('loss', t_loss)
                t_distance = distance.eval(feed_dict)
                trn_loss.append(t_loss)
                t_acc_cpu = accuracy_cpu(batch_ref, t_distance)
                trn_acc.append(t_acc_cpu)
                t_dist.append(t_distance)             

        # Evaluating model on validation,
        val_loss, val_acc, v_dist = [], [], []
        for _ in range(0, num_of_val_batches):
            batch_t0, batch_t1, batch_ref = next(valid_gen_batch)
            if batch_t0.shape[0] != batch_size:
                continue
            feed_dict={img_a: batch_t0, img_b: batch_t1, y_true: batch_ref}
            with sess.as_default():
                v_loss = loss.eval(feed_dict)
                v_distance = distance.eval(feed_dict)
                val_loss.append(v_loss)
                v_acc_cpu = accuracy_cpu(batch_ref, v_distance)
                val_acc.append(v_acc_cpu)
                v_dist.append(v_distance) 

        if best_val_loss > np.mean(val_loss):
            patience = 10
            best_val_loss = np.mean(val_loss)
            print('Saving best model and checkpoints')
            save_model(encoder, path_models +'/'+'net_enc_'+str(tm)+'.h5')
            save_model(decoder, path_models +'/'+'net_dec_'+str(tm)+'.h5')
            # Save the variables to disk.
            print('Ok')
        else:
            patience -= 1
        if patience < 0:
            break

        elapsed_time = time.time() - start_time
        print('loss trn: ', np.mean(trn_loss), 'Acc trn: ', np.mean(trn_acc), 'loss val: ', np.mean(val_loss), 'Acc val: ', np.mean(val_acc))
        [feat1_a, feat2_a, feat3_a, feat4_a] = encoder.predict(batch_t0)
        [feat1_b, feat2_b, feat3_b, feat4_b] = encoder.predict(batch_t1)        
        dist = euclidean_distance_np(feat4_a, feat4_b)
        up_distance = decoder.predict([dist, feat1_a, feat2_a, feat3_a, feat1_b, feat2_b, feat3_b])
        plot_images(batch_t0, batch_t1, up_distance, batch_ref)
    
    metrics_all.append(elapsed_time)

np.save(path_exp+'/metrics_tr.npy', metrics_all)

In [ ]:
metrics_ts = []
n_pool = 3
n_rows = 7
n_cols = 5
rows, cols = image_array.shape[:2]
pad_rows = rows - np.ceil(rows/(n_rows*2**n_pool))*n_rows*2**n_pool
pad_cols = cols - np.ceil(cols/(n_cols*2**n_pool))*n_cols*2**n_pool
print(pad_rows, pad_cols)

npad = ((0, int(abs(pad_rows))), (0, int(abs(pad_cols))), (0, 0))
image1_pad = np.pad(image_array, pad_width=npad, mode='reflect')

h, w, c = image1_pad.shape
patch_size_rows = h//n_rows
patch_size_cols = w//n_cols
num_patches_x = int(h/patch_size_rows)
num_patches_y = int(w/patch_size_cols)

new_enc = build_base_encoder((patch_size_rows, patch_size_cols, channels//2))
new_dec = build_base_decoder((patch_size_rows, patch_size_cols, 1))

In [ ]:
metrics_all =[]

for tm in range(0,times):
    print('time: ', tm)
    
    model = load_model(path_models+'/'+'net_enc_'+str(tm)+'.h5', compile=False)
    for l in range(1, len(model.layers)):
        new_enc.layers[l].set_weights(model.layers[l].get_weights())

    dec_model = load_model(path_models +'/'+'net_dec_'+str(tm)+'.h5', compile=False)
    for l in range(1, len(dec_model.layers)):
        new_dec.layers[l].set_weights(dec_model.layers[l].get_weights())
    
    start_test = time.time()
    patch_t = []

    for i in range(0,num_patches_y):
        for j in range(0,num_patches_x):
            patch = image1_pad[patch_size_rows*j:patch_size_rows*(j+1), patch_size_cols*i:patch_size_cols*(i+1), :]
            [pred1_a, pred2_a, pred3_a, pred4_a] = new_enc.predict(np.expand_dims(patch[:,:,:channels//2], axis=0))
            [pred1_b, pred2_b, pred3_b, pred4_b] = new_enc.predict(np.expand_dims(patch[:,:,channels//2:], axis=0))
            dist = euclidean_distance_np(pred4_a, pred4_b)
            up_distance = new_dec.predict([dist, pred1_a, pred2_a, pred3_a, pred1_b, pred2_b, pred3_b])
            print(up_distance.shape)
            del patch 
            patch_t.append(up_distance[:,:,:,0])
            del dist
    ts_time =  time.time() - start_test
    metrics_all.append(ts_time)
    patches_pred = np.asarray(patch_t).astype(np.float32)
    
    prob_recontructed = pred_reconctruct(h, w, num_patches_x, num_patches_y, patch_size_rows, patch_size_cols, patches_pred)
    np.save(path_maps+'/'+'prob_'+str(tm)+'.npy',prob_recontructed) 

    del prob_recontructed, model, patches_pred
metrics_ = np.asarray(metrics_all)
np.save(path_exp+'/metrics_ts.npy', metrics_)

In [ ]:
prob_rec = np.zeros((image1_pad.shape[0],image1_pad.shape[1], times))

for tm in range (0, times):
    print(tm)
    prob_rec[:,:,tm] = np.load(path_maps+'/'+'prob_'+str(tm)+'.npy').astype(np.float32)

mean_prob = np.mean(prob_rec, axis = -1)
np.save(path_maps+'/prob_mean.npy', mean_prob)

In [ ]:
# Plot mean map and reference
fig = plt.figure(figsize=(10,15))
ax1 = fig.add_subplot(121)
plt.title('Prediction')
plt.imshow(mean_prob, cmap ='jet')
ax1.axis('off')

ax2 = fig.add_subplot(122)
plt.title('Reference')
plt.imshow(ref_2019, cmap ='jet')
ax2.axis('off')

In [ ]:
# Computing metrics
mean_prob = mean_prob[:final_mask1.shape[0], :final_mask1.shape[1]]
ref1 = np.ones_like(final_mask1).astype(np.float32)

ref1 [final_mask1 == 2] = 0
TileMask = mask_amazon_ts * ref1
GTTruePositives = final_mask1==1
    
Npoints = 50
Pmax = np.max(mean_prob[GTTruePositives * TileMask ==1])
Pmin = np.min(mean_prob[GTTruePositives * TileMask ==1])
ProbList = np.linspace(Pmax,Pmin,Npoints)
 
metrics_ = matrics_AA_recall(ProbList, mean_prob, final_mask1, mask_amazon_ts, 625)
np.save(path_exp+'/acc_metrics.npy',metrics_)

In [ ]:
#  Complete NaN values
metrics_copy = metrics_.copy()
metrics_copy = complete_nan_values(metrics_copy)

In [ ]:
# Comput Mean Average Precision (mAP) score 
Recall = metrics_copy[:,0]
Precision = metrics_copy[:,1]
AA = metrics_copy[:,2]
    
DeltaR = Recall[1:]-Recall[:-1]
AP = np.sum(Precision[:-1]*DeltaR)
print('mAP', AP)

# Plot Recall vs. Precision curve
plt.close('all')
plt.plot(metrics_copy[:,0],metrics_copy[:,1])
plt.plot(metrics_copy[:,0],metrics_copy[:,2])
plt.grid()